In [ ]:
using CSV
using DataFrames
using PyCall

include("../src/Kraft.jl")

pd = pyimport("pandas")
kraft = pyimport("kraft")

In [ ]:
gene_x_sample_gct_file_path = ""

gene_x_sample_tsv_file_path = ""

CSV.write(
    gene_x_sample_tsv_file_path,
    Kraft.read(gene_x_sample_gct_file_path, "Gene");
    delim = '	',
)

gmt_file_paths = [
    "c1.all.v6.2.symbols.gmt",
    "c2.all.v6.2.symbols.gmt",
    "c3.all.v6.2.symbols.gmt",
    "c5.all.v6.2.symbols.gmt",
    "c6.all.v6.2.symbols.gmt",
    "c7.all.v6.2.symbols.gmt",
    "h.all.v6.2.symbols.gmt",
]

output_directory_path = "/Users/kwatme/Downloads/gsea"

gene_set_keywords = [
    "VANTVEER_BREAST_CANCER_ESR1",
    "DOANE_BREAST_CANCER_ESR1",
    "YANG_BREAST_CANCER_ESR1",
    "CHARAFE_BREAST_CANCER_LUMINAL_VS_BASAL",
    "AIGNER_ZEB1_TARGETS",
    "SANSOM_APC_TARGETS",
    "BCAT_GDS748",
    "BCAT.100_UP.V1",
    "PID_WNT_SIGNALING_PATHWAY",
    "LIU_CDX2_TARGETS",
    "KEGG_WNT_SIGNALING_PATHWAY",
];

In [ ]:
gene_set_x_sample_tsv_file_path = Kraft.gsea(
    gene_x_sample_tsv_file_path,
    gmt_file_paths,
    output_directory_path;
    sample_normalization_method = "-0-",
    gene_set_keywords = gene_set_keywords,
)

gene_set_x_sample = CSV.read(gene_set_x_sample_tsv_file_path)

In [ ]:
feature_x_sample = Kraft.read("/Users/kwatme/Downloads/Achilles_gene_effect.gct", "Feature")

target_row_indices = findall(feature_x_sample[!, 1] .== "Feature Name")

if length(target_row_indices) != 1

    error("there should be only 1 target.")

end

target_row_index = target_row_indices[1]

target = pd.Series(
    [feature_x_sample[target_row_index, i] for i = 2:size(feature_x_sample, 2)],
    index = names(feature_x_sample)[2:end],
    name = "Target",
)

In [ ]:
data = pd.DataFrame(
    Dict(name => gene_set_x_sample[!, name] for name in names(gene_set_x_sample)[2:end]),
    index = gene_set_x_sample[!, 1],
)

In [ ]:
kraft.make_match_panel(target, data, n_sampling = 0, n_permutation = 0)